# China Experiment

This is an attempt to replicate the China forecast from Predicting Politics

In [1]:
import copy
from collections import defaultdict, namedtuple
from itertools import combinations
import csv

import numpy as np
import scipy.stats

import pandas as pd
import networkx as nx

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('bmh')

In [2]:
from negotiation_model import *
from bdm_agent import *

In [3]:
class BDMActor(NegotiationActor):
    DecisionClass = BDM_Agent

## Load data

In [4]:
data = pd.read_csv("/Users/dmasad/Programming/Learning/BDM/ChinaData.csv")

In [5]:
data

,Stakeholder,Resources,Position,Salience
0,Zeming,80,25,50
1,Lipeng,90,15,80
2,Qiaoshi,85,15,80
3,Shanghai,30,35,50
4,Ruihuan,60,30,75
5,Tianjiyun,50,20,40
6,Zhurongji,60,35,80
7,Zoujiahua,80,15,75
8,Shangkun,100,20,85
9,Baibing,85,15,85


In [6]:
# Normalize
#data.Resources /= data.Resources.sum()
data.Resources /= 100
data.Position = ((data.Position - data.Position.min()) 
                 / (data.Position.max() - data.Position.min()))
data.Salience /= 100

## Start and run model

In [7]:
agents = []
for i, row in data.iterrows():
    agent = BDMActor(row.Stakeholder, row.Resources, row.Position, row.Salience)
    agent.decision_model.Q = 1
    agent.decision_model.T = 1
    agent.decision_model.verbose = False
    agents.append(agent)
    
model = NegotiationModel(agents)

In [8]:
model.find_median()

0.1111111111111111

In [9]:
for agent in model.agents:
    print(agent)

Zeming              	Position: 0.17	Capability: 0.80	Salience: 0.50
Lipeng              	Position: 0.06	Capability: 0.90	Salience: 0.80
Qiaoshi             	Position: 0.06	Capability: 0.85	Salience: 0.80
Shanghai            	Position: 0.28	Capability: 0.30	Salience: 0.50
Ruihuan             	Position: 0.22	Capability: 0.60	Salience: 0.75
Tianjiyun           	Position: 0.11	Capability: 0.50	Salience: 0.40
Zhurongji           	Position: 0.28	Capability: 0.60	Salience: 0.80
Zoujiahua           	Position: 0.06	Capability: 0.80	Salience: 0.75
Shangkun            	Position: 0.11	Capability: 1.00	Salience: 0.85
Baibing             	Position: 0.06	Capability: 0.85	Salience: 0.85
Wangzhen            	Position: 0.06	Capability: 0.85	Salience: 0.80
Wanli               	Position: 0.17	Capability: 0.60	Salience: 0.60
Ziannian            	Position: 0.11	Capability: 0.60	Salience: 0.60
Cheyun              	Position: 0.00	Capability: 1.00	Salience: 0.85
Boyibo              	Position: 0.11	Capability: 

In [10]:
model.step()

In [11]:
for agent in model.agents:
    print(agent)

Zeming              	Position: 0.17	Capability: 0.80	Salience: 0.50
Lipeng              	Position: 0.06	Capability: 0.90	Salience: 0.80
Qiaoshi             	Position: 0.06	Capability: 0.85	Salience: 0.80
Shanghai            	Position: 0.28	Capability: 0.30	Salience: 0.50
Ruihuan             	Position: 0.18	Capability: 0.60	Salience: 0.75
Tianjiyun           	Position: 0.11	Capability: 0.50	Salience: 0.40
Zhurongji           	Position: 0.22	Capability: 0.60	Salience: 0.80
Zoujiahua           	Position: 0.06	Capability: 0.80	Salience: 0.75
Shangkun            	Position: 0.11	Capability: 1.00	Salience: 0.85
Baibing             	Position: 0.06	Capability: 0.85	Salience: 0.85
Wangzhen            	Position: 0.06	Capability: 0.85	Salience: 0.80
Wanli               	Position: 0.17	Capability: 0.60	Salience: 0.60
Ziannian            	Position: 0.11	Capability: 0.60	Salience: 0.60
Cheyun              	Position: 0.03	Capability: 1.00	Salience: 0.85
Boyibo              	Position: 0.11	Capability: 

In [12]:
model.find_median()

0.1111111111111111

In [13]:
for agent in model.agents:
    print(agent)
    print(agent.decision_model.action_log)
    print("")

Zeming              	Position: 0.17	Capability: 0.80	Salience: 0.50
['Zeming conflict with Europe']

Lipeng              	Position: 0.06	Capability: 0.90	Salience: 0.80
['Lipeng conflict with Tianjiyun']

Qiaoshi             	Position: 0.06	Capability: 0.85	Salience: 0.80
['Qiaoshi conflict with Tianjiyun']

Shanghai            	Position: 0.28	Capability: 0.30	Salience: 0.50
['Shanghai conflict with Cheyun']

Ruihuan             	Position: 0.18	Capability: 0.60	Salience: 0.75
['Ruihuan compromise with Lipeng']

Tianjiyun           	Position: 0.11	Capability: 0.50	Salience: 0.40
['Tianjiyun conflict with Europe']

Zhurongji           	Position: 0.22	Capability: 0.60	Salience: 0.80
['Zhurongji compromise with Lipeng']

Zoujiahua           	Position: 0.06	Capability: 0.80	Salience: 0.75
['Zoujiahua conflict with Tianjiyun']

Shangkun            	Position: 0.11	Capability: 1.00	Salience: 0.85
['Shangkun conflict with Lipeng']

Baibing             	Position: 0.06	Capability: 0.85	Salience: 

### With position changes

In [29]:
class BDM_Agent_Moves_On_Challenge(BDM_Agent):
    def lose_conflict(self, winner):
        self.position = winner.position

class BDM_Moving_On_Conflict_Actor(NegotiationActor):
    DecisionClass = BDM_Agent_Moves_On_Challenge

class NegotiationModel_DeterministicWinner(NegotiationModel):
    def resolve_conflict(self, side_1, side_2):
        eu_1 = side_1.decision_model.compute_eu(side_1, side_2)
        eu_2 = side_2.decision_model.compute_eu(side_2, side_1)
        if eu_1 > eu_2:
            return True
        else:
            return False

In [30]:
agents = []
for i, row in data.iterrows():
    agent = BDM_Moving_On_Conflict_Actor(row.Stakeholder, 
                                         row.Resources, row.Position, row.Salience)
    agent.decision_model.Q = 0.5
    agent.decision_model.T = 0.5
    agent.decision_model.verbose = False
    agents.append(agent)
    
model = NegotiationModel_DeterministicWinner(agents)

In [33]:
model.step()

In [34]:
model.find_median()

0.1111111111111111

In [44]:
for agent in model.agents:
    print(agent)

Zeming              	Position: 0.17	Capability: 0.80	Salience: 0.50
Lipeng              	Position: 0.06	Capability: 0.90	Salience: 0.80
Qiaoshi             	Position: 0.06	Capability: 0.85	Salience: 0.80
Shanghai            	Position: 0.28	Capability: 0.30	Salience: 0.50
Ruihuan             	Position: 0.17	Capability: 0.60	Salience: 0.75
Tianjiyun           	Position: 0.11	Capability: 0.50	Salience: 0.40
Zhurongji           	Position: 0.19	Capability: 0.60	Salience: 0.80
Zoujiahua           	Position: 0.06	Capability: 0.80	Salience: 0.75
Shangkun            	Position: 0.11	Capability: 1.00	Salience: 0.85
Baibing             	Position: 0.06	Capability: 0.85	Salience: 0.85
Wangzhen            	Position: 0.06	Capability: 0.85	Salience: 0.80
Wanli               	Position: 0.17	Capability: 0.60	Salience: 0.60
Ziannian            	Position: 0.11	Capability: 0.60	Salience: 0.60
Cheyun              	Position: 0.06	Capability: 1.00	Salience: 0.85
Boyibo              	Position: 0.11	Capability: 

In [45]:
model.agent_names["USA"].decision_model.action_log

['USA compromise with Ruihuan', 'USA compromise with Cheyun']

In [46]:
model.find_mean()

0.13234238349526278